### Deploy Web App on Azure Container Services (AKS)
In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored.
    
The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create resource group and create AKS](#section3)
* [Connect to AKS](#section4)
* [Deploy our app](#section5)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

In [1]:
import json
from testing_utilities import write_json_to_file
from dotenv import set_key, get_key, find_dotenv

<a id='section1'></a>
## Setup
Below are the various name definitions for the resources needed to setup AKS.

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [3]:
set_key(env_path, 'selected_subscription', '<YOUR_AZURE_SUBSCRIPTION>') # Replace YOUR_AZURE_SUBSCRIPTION
set_key(env_path, 'resource_group', 'msaksrg')
set_key(env_path, 'aks_name', 'msaks')
set_key(env_path, 'location', 'eastus')

(True, 'location', 'eastus')

In [5]:
image_name = get_key(env_path, 'docker_login') + '/' +get_key(env_path, 'image_repo') 

<a id='section2'></a>
## Azure account login
If you are not already logged in to an Azure account, the command below will initiate a login. It will pop up a browser where you can select an Azure account.

In [ ]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

In [8]:
!az account set --subscription "{get_key(env_path, 'selected_subscription')}"

In [ ]:
!az account show

In [10]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [ ]:
!az provider show -n Microsoft.ContainerService

<a id='section3'></a>
## Create resource group and create AKS

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [12]:
!az group create --name {get_key(env_path, 'resource_group')} \
                 --location {get_key(env_path, 'location')}

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "msaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier. This can take up to 15 minutes.

In [13]:
!az aks create --resource-group {get_key(env_path, 'resource_group')}  \
               --name {get_key(env_path, 'aks_name')} \
               --node-count 1 \
               --generate-ssh-keys \
               -s Standard_NC6

{- Finished ..
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "count": 1,
      "maxPods": 110,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "msaks-msaksrg-edf507",
  "enableRbac": true,
  "fqdn": "msaks-msaksrg-edf507-8605c544.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msaksrg/providers/Microsoft.ContainerService/managedClusters/msaks",
  "kubernetesVersion": "1.9.9",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDCeJ/GxOdQ5tLMRzxbsSHcEvNLtaeKwVvuSltVQhgJ9oPOduM1IC6+rlsDpvewYcx0c5O7ZbnkWJGCvVnzrC2II6lJTH9WkRKaRitk1R6GJk3FkGEm59reDn1OS+BYAGmuLmDiVfXxe5eAI9aM4ejbnx8GLl3PmCrNvBzSGi/mnyE3MVu3lhim0s7MMBLpnMpWOjXAU1eR6FtyNjav++6Qon

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo az aks install-cli

<a id='section4'></a>
## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [ ]:
!az aks get-credentials --resource-group {get_key(env_path, 'resource_group')}\
                        --name {get_key(env_path, 'aks_name')}

Let's verify connection by listing the nodes. 

**NOTE: If you get an error below try deleting the .kube/config file in your home directory and running the command above again**

In [19]:
!kubectl get nodes

NAME                       STATUS   ROLES   AGE     VERSION
aks-nodepool1-24684105-0   Ready    agent   9m57s   v1.9.9


Let's check the pods on our cluster.

In [20]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY   STATUS    RESTARTS   AGE
kube-system   heapster-6f4d8b48bc-h5grr               2/2     Running   0          9m56s
kube-system   kube-dns-v20-7d874cb9b6-jjjs4           3/3     Running   0          9m56s
kube-system   kube-dns-v20-7d874cb9b6-nxg7t           3/3     Running   0          9m55s
kube-system   kube-proxy-bcwqt                        1/1     Running   0          9m53s
kube-system   kube-svc-redirect-nwsmh                 2/2     Running   0          9m54s
kube-system   kubernetes-dashboard-68f8cc4d8c-992qp   1/1     Running   2          9m55s
kube-system   tunnelfront-5fb57fc48d-wxb7g            1/1     Running   0          9m53s


<a id='section5'></a>
## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.


In [21]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                            "mountPath": "/usr/local/nvidia",
                            "name": "nvidia"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [22]:
write_json_to_file(app_template, 'az-dl.json') # We write the service template to the json file

In [23]:
write_json_to_file(service_temp, 'az-dl.json', mode='a') # We add the loadbelanacer template to the json file

Let's check the manifest created.

In [24]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": "masalvar/pytorch-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
             

Next, we will use kubectl create command to deploy our application.

In [25]:
!kubectl create -f az-dl.json

deployment.apps/azure-dl created
service/azure-dl created


Let's check if the pod is deployed.

In [34]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY   STATUS    RESTARTS   AGE
default       azure-dl-75d565c95b-cxp7w               1/1     Running   0          7m26s
kube-system   heapster-6f4d8b48bc-h5grr               2/2     Running   0          19m
kube-system   kube-dns-v20-7d874cb9b6-jjjs4           3/3     Running   0          19m
kube-system   kube-dns-v20-7d874cb9b6-nxg7t           3/3     Running   0          18m
kube-system   kube-proxy-bcwqt                        1/1     Running   0          18m
kube-system   kube-svc-redirect-nwsmh                 2/2     Running   0          18m
kube-system   kubernetes-dashboard-68f8cc4d8c-992qp   1/1     Running   2          18m
kube-system   tunnelfront-5fb57fc48d-wxb7g            1/1     Running   0          18m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [35]:
!kubectl get events

LAST SEEN   TYPE     REASON                    KIND         MESSAGE
20m         Normal   Starting                  Node         Starting kubelet.
19m         Normal   NodeHasSufficientDisk     Node         Node aks-nodepool1-24684105-0 status is now: NodeHasSufficientDisk
19m         Normal   NodeHasSufficientMemory   Node         Node aks-nodepool1-24684105-0 status is now: NodeHasSufficientMemory
19m         Normal   NodeHasNoDiskPressure     Node         Node aks-nodepool1-24684105-0 status is now: NodeHasNoDiskPressure
20m         Normal   NodeAllocatableEnforced   Node         Updated Node Allocatable limit across pods
19m         Normal   RegisteredNode            Node         Node aks-nodepool1-24684105-0 event: Registered Node aks-nodepool1-24684105-0 in Controller
18m         Normal   Starting                  Node         Starting kube-proxy.
7m29s       Normal   Scheduled                 Pod          Successfully assigned azure-dl-75d565c95b-cxp7w to aks-nodepool1-24684105-0

In [36]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-10-08 14:11:45,238 CRIT Supervisor running as root (no user in config file)
2018-10-08 14:11:45,240 INFO supervisord started with pid 1
2018-10-08 14:11:46,242 INFO spawned: 'program_exit' with pid 10
2018-10-08 14:11:46,243 INFO spawned: 'nginx' with pid 11
2018-10-08 14:11:46,244 INFO spawned: 'gunicorn' with pid 12
2018-10-08 14:11:47,271 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
77.4%2018-10-08 14:11:51,263 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
100.0%{"timestamp": "2018-10-08T14:11:55.439918Z", "message": "Model loading time: 8668.95 ms", "host": "azure-dl-75d565c95b-cxp7w", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "model_driver", "stack_info": null}
Initialising
{"timestamp": "2018-10-08T14:11:55.451379Z", "messa

It can take a few minutes for the service to populate the EXTERNAL-IP field. This will be the IP you use to call the service. You can also specify an IP to use please see the AKS documentation for further details.

In [37]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.188.145   40.114.88.243   80:30494/TCP   7m45s


Next, we will [test our web application](05_TestWebApp.ipynb) deployed on AKS. 